In [4]:
import pickle
import pandas as pd
from pathlib import Path
import networkx as nx
import py4cytoscape as p4c

In [5]:
with open("/Users/daffa/Documents/Work/jspp67_bioinf/results/gene_network_backup/sbi_G_annotated-z4_k10.pkl", "rb") as f:
    G = pickle.load(f)

In [7]:
# basic graph stats
print(G.number_of_nodes(), "nodes")
print(G.number_of_edges(), "edges")

20006 nodes
114053 edges


In [8]:
# check node annotations
import pandas as pd
node_data = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient="index")
node_data["pathway"].value_counts()

pathway
background    19978
map00020         23
map00660          5
Name: count, dtype: int64

# visualizing graph

In [9]:

# --- 1. Build subgraph for map00020 and map00660 ---
pathways_of_interest = {"map00020", "map00660"}

seed_nodes = [
    n for n, d in G.nodes(data=True)
    if d.get("pathway") in pathways_of_interest
]

# include 1-hop neighbours so context edges are visible
neighbour_nodes = set(seed_nodes)
for n in seed_nodes:
    neighbour_nodes.update(G.neighbors(n))

H = G.subgraph(neighbour_nodes).copy()

# tag neighbour-only nodes so we can style them differently
for n in H.nodes():
    if H.nodes[n].get("pathway") not in pathways_of_interest:
        H.nodes[n]["pathway"] = "context"

print(f"Subgraph: {H.number_of_nodes()} nodes, {H.number_of_edges()} edges")


Subgraph: 397 nodes, 1145 edges


In [10]:

# --- 2. Send subgraph to Cytoscape ---
p4c.cytoscape_ping()   # make sure Cytoscape is running

network_suid = p4c.create_network_from_networkx(
    H,
    title="map00020 & map00660 clusters",
    collection="Gene Network"
)
print(f"Network created – SUID: {network_suid}")


You are connected to Cytoscape!
Applying default style...


In commands_post(): No network views selected.


Applying preferred layout
Network created – SUID: 128


In [11]:

# --- 3. Apply visual style ---
style_name = "pathway_style"

# create a blank style
p4c.create_visual_style(style_name)

# node fill colour mapped to the 'pathway' attribute
pathway_colours = {
    "map00020": "#E41A1C",   # red  – TCA cycle
    "map00660": "#377EB8",   # blue – propanoate metabolism
    "context":  "#CCCCCC",   # grey – neighbour context nodes
}

p4c.set_node_color_mapping(
    table_column="pathway",
    table_column_values=list(pathway_colours.keys()),
    colors=list(pathway_colours.values()),
    mapping_type="d",        # discrete
    style_name=style_name,
    network=network_suid,
)

# make seed nodes (map00020 / map00660) larger
p4c.set_node_size_mapping(
    table_column="pathway",
    table_column_values=["map00020", "map00660", "context"],
    sizes=[60, 60, 30],
    mapping_type="d",
    style_name=style_name,
    network=network_suid,
)

# use gene_id as the node label
p4c.set_node_label_mapping(
    table_column="name",
    style_name=style_name,
    network=network_suid,
)

p4c.set_visual_style(style_name, network=network_suid)
print("Visual style applied.")


Visual style applied.


In [12]:

# --- 4. Apply layout and fit view ---
p4c.layout_network("force-directed", network=network_suid)
p4c.fit_content(network=network_suid)
print("Layout applied. Check Cytoscape.")


Layout applied. Check Cytoscape.
